In [10]:
import sys
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import numpy as np
import itertools
from itertools import product
import pprint
import sys
import functools as ft
from sklearn.utils import shuffle

sys.path.append('Code/')
from single_split_gridsearch import SingleSplitGridSearch


In [15]:

def get_final_predictions(n_patches, predicted):
    songs = int(predicted.shape[0] / n_patches)

    r = predicted.reshape((songs,n_patches))
    r = [np.unique(i, return_counts=True) for i in r]
    r = [ i[0][np.argmax(i[1])] for i in r]
    
    return r

def get_final_predictions_sumvote(n_patches, predicted):
    songs = int(predicted.shape[0] / n_patches)
    r = predicted.reshape((songs, n_patches,-1))    
    return np.argmax(np.sum(r, 1), -1)  + 1

def get_labels(n_patches, labels):
    o = []
    for i in range(0,len(labels),n_patches):
        o.append(labels[i])
    return np.array(o)


def run_experiment(validation_test_size, folds, total_patches, proto_x, proto_y, njobs):

    results = {}
        
    for patches in total_patches:
        print("patches: %d" % (patches))
        results[patches] = {'max_f1' : [], 'sum_f1' : [], 'max_acc' : [], 'sum_acc' : []}

        for test_fold in folds:
            print("fold: %d" % (test_fold))

            train_folds = sorted(list(set(folds) - set([test_fold])))
            train_X = np.concatenate([np.load(proto_x % (f,patches)) for f in train_folds])
            train_Y = np.concatenate([np.load(proto_y % (f,patches)) for f in train_folds])
            
            test_X = np.load(proto_x % (test_fold,patches))
            test_Y = np.load(proto_y % (test_fold,patches))

            ss = StandardScaler()
            train_X = ss.fit_transform(train_X)
            test_X = ss.transform(test_X)


            params = {'C' : [1, 10, 100, 1000], 'gamma' : ['auto', 2e-1, 2e-2, 2e-3]}
            svm = SVC(probability=True)

            clf = SingleSplitGridSearch(svm, params, n_jobs=njobs, test_size=validation_test_size)
            print('train_X {} : Train_y {}'.format(len(train_X), len(train_Y)))
            print('test_X {} : test_Y {}'.format(len(test_X), len(test_Y)))


            clf.fit(train_X, train_Y)

            print (clf.best_params_)

            predicted = clf.predict(test_X)
            
            
            prob = clf.best_estimator_.predict_proba(test_X)

            l = get_labels(patches, test_Y)

            p = get_final_predictions(patches, predicted)
            acc, f1 = accuracy_score(l, p), f1_score(l, p, average='weighted')
            results[patches]['max_acc'].append(acc)
            results[patches]['max_f1'].append(f1)
            print("MAX: acc: %.3f f1: %.3f" % (acc, f1))
            
            p = get_final_predictions_sumvote(patches, prob)
            acc, f1 = accuracy_score(l, p), f1_score(l, p, average='weighted')
            
            print("\n\n\n----------------------------")
            print("Predictions    : ", p)
            print("\n\nLabels Corretas:   ", l)
            print("----------------------------\n\n\n")
            
            results[patches]['sum_acc'].append(acc)
            results[patches]['sum_f1'].append(f1)  
            print("SUM: acc: %.3f f1: %.3f" % (acc, f1))
            
            sys.stdout.flush()
                              
    return results


In [16]:
def run():
    folds = [1, 2, 3, 4, 5]
    patches = [1, 3, 5, 10]
    
    escolhas = [ ['BSIF/', 'BSIF_Folder%d_%dx1'], ['LBP/', 'LBP_Folder%d_%dx1'], ['LPQ/','LPQ_Folder%d_%dx1'] ]
    #escolhas = [  ['MobileNet/', 'commonvoicev1_Fold%d_%dx1'], ['VGG16/', 'commonvoicev1_Fold%d_%dx1'] ]
    for choice in escolhas:
        features_folder = '../datasets/'+choice[0]


        output_proto = features_folder + choice[1]
        
        results = run_experiment(0.25, 
          folds, 
          patches, 
          output_proto + "_X.npy", 
          output_proto + "_Y.npy",njobs=8)
        
        print("results")

        np.save(f'resultados_npy_identicacao/{choice[0].replace("/", "")}_v2.npy', [results], allow_pickle=True) 

        pprint.pprint(results)

In [17]:

run()

patches: 1
fold: 1
train_X 320 : Train_y 320
test_X 80 : test_Y 80
{'C': 10, 'gamma': 'auto'}
MAX: acc: 0.338 f1: 0.292



----------------------------
Predictions    :  [71  2 30 62 48  6  6  8  9  4  6 12 42 48  2 16 17 18  6 74 46  4 23 40
 26  2  6 38 60 30 15  6 39 72  6 68 60 42 65 78  9 79 76 48 75 46 37 60
 48 76 65 74  4 75 75 56  2 58  6 63 65 63  4 32 65 69 76 68 69 42 42 75
 70  6 75 42 75 78 79 68]


Labels Corretas:    [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 78 79 80]
----------------------------



SUM: acc: 0.237 f1: 0.172
fold: 2
train_X 320 : Train_y 320
test_X 80 : test_Y 80
{'C': 10, 'gamma': 0.002}
MAX: acc: 0.463 f1: 0.376



----------------------------
Predictions    :  [71  2 13  9 78 19 80  8  9 74  6 61 37 14  2 16 66 41 19 74 78  2 77 74
 50 68  2 46 13 30 74 61 77 5

fold: 3
train_X 1600 : Train_y 1600
test_X 400 : test_Y 400
{'C': 100, 'gamma': 'auto'}
MAX: acc: 0.275 f1: 0.229



----------------------------
Predictions    :  [75  2 62 14 14 35 63  8 22 15 11 11  1 17 15 15 66 18 19 14 70 35  8 30
 29 34 22 14  8 30  2 32 40 33 34  6  1 15 58 65 41  1 43 35 25 46 46 62
 38 50 50 80 54 62 11 56 80 58 46 17 32 62 62 64 64 15 39 68 69 70  7 70
 73 74  1 54 66 35  1 80]


Labels Corretas:    [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 78 79 80]
----------------------------



SUM: acc: 0.275 f1: 0.202
fold: 4
train_X 1600 : Train_y 1600
test_X 400 : test_Y 400
{'C': 1000, 'gamma': 0.002}
MAX: acc: 0.275 f1: 0.207



----------------------------
Predictions    :  [42  2 62  4 10  6 71  8  9 74 62 11  4 14 22 16 17 18 19 30 41 27  8  6
 21 26 27 33 27 10 63 32 41 3

fold: 2
train_X 320 : Train_y 320
test_X 80 : test_Y 80
{'C': 1, 'gamma': 0.2}
MAX: acc: 0.312 f1: 0.253



----------------------------
Predictions    :  [73 73 65 73 73 73 73  8 73 73 73 73 73 73 73 73 73 73 73 73 73 73 72 73
 44 73 73 73 73 73 73 73 71 44 73  8 73 73 73 44 73 62 44 73 75 73 73 73
 73 73 72 73 73 73 72 73 75 74 80 73 62 73 73 73 72 73 62 73 73 44 73 72
 65 73 75 44 73 73 62 62]


Labels Corretas:    [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 78 79 80]
----------------------------



SUM: acc: 0.037 f1: 0.019
fold: 3
train_X 320 : Train_y 320
test_X 80 : test_Y 80
{'C': 1, 'gamma': 0.2}
MAX: acc: 0.225 f1: 0.192



----------------------------
Predictions    :  [72 79 79 62 33 62 64  8 62 62 62 64 64 64 64 62 62 62 64 33 69 62 64 64
 62 64 62 73 64 64 62 33 62 64 64 33 33 64 73 6

fold: 4
train_X 1600 : Train_y 1600
test_X 400 : test_Y 400
{'C': 10, 'gamma': 'auto'}
MAX: acc: 0.275 f1: 0.208



----------------------------
Predictions    :  [79  2 37  4  4  6  7  8  9 37 17 12 31 15  2 15 66 18 19 20 21 22  8  6
 25 70  2 63  7  6 63 32 33 72 66 14 57 32 60 21 41 42 76 66 74 46 46 66
 10 68 72 56 49 72  8 56  8 58 59 60 37 41 22 64 65 66 67 68 62 72 37 76
 76 74 75 18 72  4 37 41]


Labels Corretas:    [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 78 79 80]
----------------------------



SUM: acc: 0.362 f1: 0.280
fold: 5
train_X 1600 : Train_y 1600
test_X 400 : test_Y 400
{'C': 1000, 'gamma': 'auto'}
MAX: acc: 0.163 f1: 0.133



----------------------------
Predictions    :  [ 6  2  4 10 32  6  8  9 22 11 12 73 14  4 31 17 18 19 19 21 22 29 31 25
 67 27 28 59 74 31 32 60  5 7

fold: 3
train_X 320 : Train_y 320
test_X 80 : test_Y 80
{'C': 10, 'gamma': 0.02}
MAX: acc: 0.287 f1: 0.242



----------------------------
Predictions    :  [55  2 30  2 31 15 80  8  3  6  3 56 80 17  2 16 56 68 19 74 80 35 23 74
 30 67 16 14 23 62  2 59 78 33 67 35  1 74 46 25 18 29 65 59 55 61 50 33
 60 50 39 46 54 39 47 56  6 58  9 70 61 62  6 64 65 15 39 68 69 50 71 72
 73 14 39 54 23  3 39 80]


Labels Corretas:    [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 78 79 80]
----------------------------



SUM: acc: 0.225 f1: 0.170
fold: 4
train_X 320 : Train_y 320
test_X 80 : test_Y 80
{'C': 100, 'gamma': 0.002}
MAX: acc: 0.487 f1: 0.423



----------------------------
Predictions    :  [79  2 12  9  3 30 21  8  9 38 18 12 14 14  2 19 14 18 19 14 41  4 16 21
 25 70  2  4 30 30 14 37 42 67 10 14 21 1

fold: 5
train_X 1600 : Train_y 1600
test_X 400 : test_Y 400
{'C': 10, 'gamma': 0.002}
MAX: acc: 0.087 f1: 0.073



----------------------------
Predictions    :  [80 22  4  5 72  5  8  9  5 11 12 73 17  2 31 17 18 19 19 41  4 23 74 25
 72 27 24 42 62 31 78 58 33 72 22 64 71 65 79 41 73 70 78 45 46 78 60 47
 73 70  5 70 55 55 78 56 42 32 59 60 14 62 60 64 76 66 72 47 11 70 67 72
 73 74  8 72 61 78  7 49]


Labels Corretas:    [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 78 79 80]
----------------------------



SUM: acc: 0.163 f1: 0.107
patches: 10
fold: 1
train_X 3200 : Train_y 3200
test_X 800 : test_Y 800
{'C': 1000, 'gamma': 'auto'}
MAX: acc: 0.300 f1: 0.241



----------------------------
Predictions    :  [71 15 31 74 32 19 71  8  9 22  6 12 68 14  9 16 14  9 15 74 21 27  8 40
 58 27 69 14 77 30